In [1]:
from minerva.models.ssl.tfc import TFC_Model
from minerva.models.nets.tfc import TFC_Backbone
from minerva.models.nets.tnc import TSEncoder
import warnings
import warnings
import lightning as L
from torch.utils.data import DataLoader
from lightning.pytorch.callbacks import ModelCheckpoint
from lightning.pytorch.loggers import CSVLogger
from minerva.data.datasets.series_dataset import MultiModalSeriesCSVDataset
from torch.nn import TransformerEncoder, TransformerEncoderLayer

warnings.filterwarnings("ignore")


# os.environ["CUDA_VISIBLE_DEVICES"] = "1"

/usr/local/lib/python3.10/dist-packages/_distutils_hack/__init__.py:53: UserWarning: Reliance on distutils from stdlib is deprecated. Users must rely on setuptools to provide the distutils module. Avoid importing distutils or import setuptools first, and avoid setting SETUPTOOLS_USE_DISTUTILS=stdlib. Register concerns at https://github.com/pypa/setuptools/issues/new?template=distutils-deprecation.yml
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
batch_size = 128
input_channels = 6
TS_length = 60
num_classes = 1

model = TFC_Model(
    input_channels=input_channels,
    batch_size=batch_size,
    TS_length=TS_length,
    num_classes=None,
    batch_1_correction=True,
    backbone=TFC_Backbone(
        input_channels=input_channels,
        TS_length=TS_length,
        time_encoder=TSEncoder(
            input_dims=input_channels,
            output_dims=64,
            hidden_dims=64,
            depth=10,
            permute=True,
        ),
        frequency_encoder=TSEncoder(
            input_dims=input_channels,
            output_dims=64,
            hidden_dims=64,
            depth=10,
            permute=True,
        ),
    ),
)

model


# model = TFC_Model(input_channels=input_channels, TS_length=TS_length, batch_1_correction=True, batch_size=batch_size)

TFC_Model(
  (backbone): TFC_Backbone(
    (time_encoder): TSEncoder(
      (input_fc): Linear(in_features=6, out_features=64, bias=True)
      (feature_extractor): DilatedConvEncoder(
        (net): Sequential(
          (0): ConvBlock(
            (conv1): SamePadConv(
              (conv): Conv1d(64, 64, kernel_size=(3,), stride=(1,), padding=(1,))
            )
            (conv2): SamePadConv(
              (conv): Conv1d(64, 64, kernel_size=(3,), stride=(1,), padding=(1,))
            )
          )
          (1): ConvBlock(
            (conv1): SamePadConv(
              (conv): Conv1d(64, 64, kernel_size=(3,), stride=(1,), padding=(2,), dilation=(2,))
            )
            (conv2): SamePadConv(
              (conv): Conv1d(64, 64, kernel_size=(3,), stride=(1,), padding=(2,), dilation=(2,))
            )
          )
          (2): ConvBlock(
            (conv1): SamePadConv(
              (conv): Conv1d(64, 64, kernel_size=(3,), stride=(1,), padding=(4,), dilation=(4,))
     

In [ ]:
data_path = (
    "/workspaces/HIAAC-KR-Dev-Container/some_datasets/ES_Raw/es_full.0.csv"
)

train_dataset = MultiModalSeriesCSVDataset(
    data_path,
    feature_prefixes=[
        "accel-x",
        "accel-y",
        "accel-z",
        "gyro-x",
        "gyro-y",
        "gyro-z",
    ],
    label="standard activity code",
    features_as_channels=False,
)

train_dataset

MultiModalSeriesCSVDataset at /workspaces/HIAAC-KR-Dev-Container/some_datasets/ES_Raw/es_full.0.csv (250000 samples)

: 

In [4]:
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
train_dataloader

In [6]:
batch_x, batch_y = next(iter(train_dataloader))
batch_x.shape, batch_y.shape

(torch.Size([128, 6, 60]), torch.Size([128]))

In [8]:
callbacks = [
    ModelCheckpoint(
        filename="tfc-{epoch:02d}",
        every_n_epochs=1,
    ),
]

logger = CSVLogger(save_dir="logs", name="tfc-normal-es")

trainer = L.Trainer(
    max_epochs=100,
    accelerator="gpu",
    devices=1,
    callbacks=callbacks,
    logger=logger,
)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs


In [ ]:
# batch_x, batch_y = next(iter(train_dataloader))
# batch_x.shape, batch_y.shape

trainer.fit(
    model, train_dataloaders=train_dataloader
)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name     | Type            | Params | Mode 
-----------------------------------------------------
0 | backbone | TFC_Backbone    | 472 K  | train
1 | loss_fn  | NTXentLoss_poly | 0      | train
-----------------------------------------------------
472 K     Trainable params
0         Non-trainable params
472 K     Total params
1.890     Total estimated model params size (MB)
55        Modules in train mode
0         Modules in eval mode


Epoch 0:  31%|███       | 598/1954 [01:06<02:30,  9.01it/s, v_num=0]